In [1]:
# Устанавливаем необходимую библиотеку wordfreq
# Она содержит частотный словарь слов для русского языка
!pip install wordfreq --quiet

# Импорты
import pandas as pd
import math
import re
from wordfreq import word_frequency, top_n_list

In [2]:
# Функция для чтения исходного датасета
# Важно: делим строку только по первой запятой, т.к. внутри текста могут быть запятые
def load_dataset(path: str) -> pd.DataFrame:
    rows = []
    with open(path, encoding="utf-8") as f:
        next(f)  # пропускаем заголовок
        for line in f:
            line = line.strip()
            if not line:
                continue
            id_, text = line.split(",", 1)
            rows.append((id_, text))
    return pd.DataFrame(rows, columns=["id", "text_no_spaces"])

# Загружаем данные
task_data = load_dataset("dataset_1937770_3.txt")

# Смотрим на первые строки
task_data.head()

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку


In [3]:
# Получаем список наиболее частотных слов (топ-50k) из wordfreq
vocab = set(top_n_list("ru", 50000))

# Определяем наборы символов для обработки
PUNCT = set(",.!?:;")        # стандартная пунктуация
DASHES = {"-", "–", "—"}     # дефис и разные виды тире

In [4]:
# Разбиваем строку на токены (кириллица, латиница, цифры, знаки препинания)
def normalize_tokens(word: str):
    """
    Универсальная разбивка на токены:
    - кириллические слова
    - латиница
    - числа
    - пунктуация
    - дефисы/тире
    """
    return re.findall(r"[А-Яа-яёЁ]+|[A-Za-z]+|[0-9]+|[.,!?;:()\"'«»\-–—]", word)

# Дополнительная обработка токенов:
# 1. Разделяем склеенные цифры и буквы (128гб → 128 гб)
# 2. Разделяем латиницу и кириллицу (XiaomiТелефон → Xiaomi Телефон)
def fix_tokens(tokens):
    fixed = []
    for t in tokens:
        # Разделение "цифра+буква"
        if re.search(r"[0-9][A-Za-zА-Яа-я]", t) or re.search(r"[A-Za-zА-Яа-я][0-9]", t):
            fixed.extend(re.findall(r"[0-9]+|[A-Za-zА-Яа-я]+", t))
            continue
        # Разделение "латиница+кириллица"
        if re.search(r"[A-Za-z]+[А-Яа-яёЁ]+", t) or re.search(r"[А-Яа-яёЁ]+[A-Za-z]+", t):
            fixed.extend(re.findall(r"[A-Za-z]+|[А-Яа-яёЁ]+", t))
            continue
        # Иначе токен оставляем как есть
        fixed.append(t)
    return fixed

In [5]:
# Основная функция сегментации текста
def segment(text: str, max_word_len=20):
    n = len(text)

    # DP массивы: dp[i] = минимальная стоимость разбиения текста[0:i]
    dp = [float("inf")] * (n + 1)
    prev = [-1] * (n + 1)
    dp[0] = 0  # базовый случай

    # Перебираем все префиксы текста
    for i in range(1, n + 1):
        for j in range(max(0, i - max_word_len), i):
            word = text[j:i].lower()
            # Стоимость слова: -log(probability), либо штраф если слова нет
            prob = word_frequency(word, "ru")
            cost = -math.log(prob) if prob > 0 else 50
            if dp[j] + cost < dp[i]:
                dp[i] = dp[j] + cost
                prev[i] = j

    # Восстановление наилучшей последовательности слов
    words, i = [], n
    while i > 0:
        j = prev[i] if prev[i] != -1 else i - 1
        words.append(text[j:i])
        i = j
    words.reverse()

    # Доп. токенизация и фиксы
    tokens = []
    for w in words:
        tokens.extend(normalize_tokens(w))
    tokens = fix_tokens(tokens)

    # Проставляем пробелы
    pos, cur = [], 0
    for idx, w in enumerate(tokens[:-1]):
        cur += len(w)
        nxt = tokens[idx + 1]

        if nxt in PUNCT:
            # пунктуация прилипает к слову → пробел перед ней НЕ нужен
            continue
        elif w in PUNCT:
            # после пунктуации ставим пробел
            pos.append(cur)
        elif w in DASHES or nxt in DASHES:
            # тире/дефис → пробел до и после
            pos.append(cur)
        else:
            pos.append(cur)

    # Возвращаем позиции пробелов (индексы внутри строки без пробелов)
    return sorted(set(p for p in pos if 0 < p < len(text)))

In [6]:
# Применяем сегментацию ко всему датасету
task_data["predicted_positions"] = task_data["text_no_spaces"].apply(segment)

# Преобразуем список чисел в строку (формат "[1, 5, 10]")
task_data["predicted_positions"] = task_data["predicted_positions"].apply(str)

# Проверяем результат на первых примерах
print(task_data.head())

# Сохраняем финальный submission
task_data.to_csv("submission.csv", index=False, encoding="utf-8-sig")
print("submission.csv готов")

  id                 text_no_spaces  predicted_positions
0  0                куплюайфон14про          [5, 10, 12]
1  1             ищудомвПодмосковье            [3, 6, 7]
2  2  сдаюквартирусмебельюитехникой  [4, 12, 13, 20, 21]
3  3     новыйдивандоставканедорого          [5, 10, 18]
4  4                отдамдаромкошку              [5, 10]
submission.csv готов
